In [9]:
def read_data_columns() -> list[str]:
    import pandas as pd
    # read the columsn from excel
    # Feature,user_id,gender,visit_city,avg_price,is_supervip,ctr_30,ord_30,total_amt_30,shop_id,item_id,city_id,district_id,shop_aoi_id,shop_geohash_6,shop_geohash_12,brand_id,category_1_id,merge_standard_food_id,rank_7,rank_30,rank_90,shop_id_list,item_id_list,category_1_id_list,merge_standard_food_id_list,brand_id_list,price_list,shop_aoi_id_list,shop_geohash6_list,timediff_list,hours_list,time_type_list,weekdays_list,times,hours,time_type,weekdays,geohash12
    df = pd.read_excel('../../data/D1_0_top_10.xlsx')
    # read the sheet names
    sheet_names = pd.ExcelFile('../../data/D1_0_top_10.xlsx').sheet_names
    print("Reading Sheet list from ../../data/D1_0_top_10.xlsx:", sheet_names)
    # read data for the sheet named "columns"
    df_columns = pd.read_excel('../../data/D1_0_top_10.xlsx', sheet_name='D1_0_top10')
    columns = df_columns.loc[1] # access the second row
    return columns.tolist()


In [82]:
import pandas as pd

# read top 10000 rows, column 0 and column 2 are integers
df = pd.read_csv('../../data/raw/D1_0_top_10k.csv', header=None, nrows=10000, dtype={0: int, 2: "Int64"})
# print(df.head(1))

# # select the rows where column 2 is null
null_rows = df[df[2].isnull()]
# check the number of null rows in column 2
print("null_rows.shape = ", null_rows.shape) # return the shape (rows, columns)
# print("null_rows.count().sum() = ", null_rows.count()) # check the number of non-null rows in column 2

# count the column 0, group by column 2, including null values
print("==============================================")
count = df.groupby(2, dropna=False)[0].count()
print(count)  # display the count
print("sum = ", count.sum())  # sum should be 10000 

# sum the column 0, group by column 2, including null values
print("===== 按性别统计成功人次(column 0) ===")
sum = df.groupby(2, dropna=False)[0].sum()
print(sum)  # display the sum

print("===== 按性别统计成功概率 sum(column 0) / count(column 0) ===")
prob = sum / count
print(prob)  # display the probability


null_rows.shape =  (266, 39)
2
-99      430
1       3681
2       5623
<NA>     266
Name: 0, dtype: int64
sum =  10000
===== 按性别统计成功人次(column 0) ===
2
-99     14
1       64
2       80
<NA>     4
Name: 0, dtype: int64
===== 按性别统计成功概率 sum(column 0) / count(column 0) ===
2
-99     0.032558
1       0.017387
2       0.014227
<NA>    0.015038
Name: 0, dtype: float64


In [ ]:
# 移除 Metal 相关部分，在MACOS M1上使用并行化计算
import pandas as pd
from numba import njit, prange

df = pd.DataFrame({"a": range(1_000_000), "b": range(1_000_000)})

# 使用并行化替代 Metal GPU 加速
@njit(parallel=True)
def compute_sum(a, b):
    result = a + b
    return result

# 执行并行计算
df["sum"] = compute_sum(df["a"].values, df["b"].values)
print(df.head())

In [ ]:
## 测试MAC M1 GPU加速
import pandas as pd
from numba import njit, objmode
from numba import metal  # Metal后端

df = pd.DataFrame({"a": range(1_000_000), "b": range(1_000_000)})

# 用Numba加速自定义函数（M1 GPU执行）
@njit(target_backend="metal")
def compute_sum(a, b):
    result = a + b
    return result

# 执行GPU运算
df["sum"] = compute_sum(df["a"].values, df["b"].values)

In [ ]:
import pandas as pd
# 兼容方式：避免使用 PyArrow 字符串后端
pd.set_option("mode.string_storage", "python")
print("pandas string_storage:", pd.get_option("mode.string_storage"))

In [ ]:
# 用 Pandas 读取 大数据文件，分块处理，适合简单的数据处理，复杂的分析建议用 Dask
import pandas as pd
import glob
results = []
for file in glob.glob('../../data/raw/*.csv'):
    chunk = pd.read_csv(file, dtype={'col': 'float32'}, usecols=['key_cols'])
    results.append(chunk['col'].mean())  # 累积统计


In [ ]:
%%time

import subprocess
import dask.dataframe as dd
import os

CSV_PATH = ("../../data/raw/D1_0.csv") # 可以替换为*.csv
print("Preparing data:", CSV_PATH)
file_exists = os.path.exists(CSV_PATH)
if file_exists:
    print(f"{CSV_PATH} already exists, no need to unzip.")
else:
    subprocess.run(["gunzip", "--keep", CSV_PATH+".gz"], check=True)

print("Reading CSV file from:", CSV_PATH)
# 强制所有列为 object 类型，避免自动推断触发 PyArrow StringArray
df = dd.read_csv(
            str(CSV_PATH),
            header = None,
            names = read_data_columns(),
            sep=",",
            encoding="utf-8",  # 如果报编码错，试试 "latin1"
            dtype=str,  # 先全部读为字符串，保证能读进来
            assume_missing=True,
            blocksize="128MB", # for gzip files using: blocksize=None,
            sample=256_000,  # 更小样本，减少不稳定推断
            on_bad_lines="warn",  # 或 "skip" 跳过坏行
            engine="python",  # Python引擎更宽容，但速度较慢
            # quotechar='"',         # 如果有引号且格式复杂，保留或按需调整
            # na_filter=False,       # 如不希望将特定字符串视为 NA，可关闭
        )
# print(df.tail(1))  # 输出最后一行  # 仅触发首块，验证能构成 DataFrame建
print("===============================================================")
# print(f"Second column of the first row: {df.iloc[0, 1]}")
# 统计总行数
nparts = df.npartitions
total_rows = df.shape[0].compute()
print(f"Number of partitions: {nparts}, Total rows: {total_rows}")
print("===============================================================")

df['gender'].compute().value_counts(dropna=False)  # 计算性别列的值计数，包括缺失值, elapse: CPU times: user 3min 35s, sys: 38.5 s, total: 4min 14s
# df['gender'].value_counts(dropna=False).compute()  # 计算性别列的值计数，包括缺失值, elapse: CPU times: user 3min 34s, sys: 28.9 s, total: 4min 3s

Preparing data: ../../data/raw/D1_0.csv
../../data/raw/D1_0.csv already exists, no need to unzip.
Reading CSV file from: ../../data/raw/D1_0.csv
Reading Sheet list from ../../data/D1_0_top_10.xlsx: ['D1_0_top10', 'columns']
Number of partitions: 128, Total rows: 2170299
CPU times: user 3min 35s, sys: 38.5 s, total: 4min 14s
Wall time: 4min 9s


gender
2       1122603
1        760600
-99      184221
<NA>     102875
Name: count, dtype: int64[pyarrow]

In [7]:
%%time
print("Hello World")
print("This is a test cell.")

Hello World
This is a test cell.
CPU times: user 53 μs, sys: 4 μs, total: 57 μs
Wall time: 56 μs
